In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from catboost import CatBoostClassifier
#sys.setrecursionlimit(2000)
import sys
sys.setrecursionlimit(2000)

In [ ]:
df=pd.read_csv('../input/datazs.csv')

In [ ]:
df1=df.drop('Unnamed: 0',axis=1)
df1

In [ ]:
df1['type_of_combined_shot'].fillna(df1['type_of_shot'],inplace=True)

In [ ]:
df1.isnull().sum()
df1.drop('type_of_shot',axis=1,inplace=True)

In [ ]:
#df1['power_of_shot'].fillna(df1['power_of_shot'].mean,inplace=True)
#df1['distance_of_shot'].fillna(df1['distance_of_shot'].mean,inplace=True)
df1.isnull().sum()

In [ ]:
df1['area_of_shot'].fillna(method='ffill', inplace = True)
df1['shot_basics'].fillna(method='ffill', inplace = True)
df1['range_of_shot'].fillna(method='ffill', inplace = True)
df1['game_season'].fillna(method='ffill', inplace = True)
df1['home/away'].fillna(method='ffill', inplace = True)
df1['team_name'].fillna(method='ffill', inplace = True)
df1['remaining_min'].fillna(method='ffill', inplace = True)
df1['remaining_sec'].fillna(method='ffill', inplace = True)
df1['power_of_shot'].fillna(method='ffill', inplace = True)
df1['distance_of_shot'].fillna(method='ffill', inplace = True)
df1['date_of_game'].fillna(method='ffill', inplace = True)
df1['location_x'].fillna(method='ffill', inplace = True)
df1['location_y'].fillna(method='ffill', inplace = True)
df1['knockout_match'].fillna(method='ffill', inplace = True)
df1['home/away'].fillna(method='ffill', inplace = True)
#df1['lat/lng'].fillna(met)

In [ ]:
df2=df1

In [ ]:
df2['lat/lng'].fillna(df['lat/lng'].mean,inplace =True)
df2

In [ ]:
df2.isnull().sum()

In [ ]:
df3=df2.drop(['match_event_id','remaining_min.1','power_of_shot.1','knockout_match.1','remaining_sec.1','distance_of_shot.1'],axis=1)

In [ ]:
df3

In [ ]:
df3.isnull().sum()

In [ ]:
test_set = df3[df3.is_goal.isnull()]

#test_set.drop('is_goal',axis=1,inplace=True)
#data = test_set[test_set['shot_id_number'].notnull()]

submission=test_set[['shot_id_number','is_goal']]
test_set.columns
#test_set


In [ ]:
test_set=test_set.drop('is_goal',axis=1)

In [ ]:
test_set=test_set.drop('shot_id_number',axis=1)

In [ ]:
test_set=test_set.drop(['match_id','team_id','team_name'],axis=1)

In [ ]:
test_set.columns
submission.columns

In [ ]:
df3.dropna(subset=['is_goal'],inplace=True)

In [ ]:
len(df3)

In [ ]:
len(test_set)
y=df3['is_goal']
len(y)

In [ ]:
df3.drop(['shot_id_number','is_goal'],axis=1,inplace=True)

In [ ]:
df3.columns

In [ ]:
df3=df3.drop(['match_id','team_id','team_name'],axis=1)

In [ ]:
df3.columns

In [ ]:
#test_set.drop('is_goal',axis=1,inplace=True)
test_set.columns

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df3, y, test_size = 0.25, random_state = 0)

In [ ]:
model = CatBoostClassifier(iterations=1500, learning_rate=0.08, l2_leaf_reg=3.5, depth=9, rsm=0.98, loss_function= 'Logloss',eval_metric='MAE',use_best_model=True,random_seed=42)

In [ ]:
df3.columns

In [ ]:
cate_features_index = np.where(df3.dtypes != float)
myarray= np.asarray(cate_features_index)
myarray=myarray.astype(int)
myarray.dtype

In [ ]:
#model.fit(X_train,y_train,)
#model.fit(X_train,y_train,cat_features=myarray,eval_set=(X_test,y_test))

In [ ]:
df1.isnull().sum()

In [ ]:
dfenc=df1.drop(['team_id','match_id','match_event_id','remaining_min.1','power_of_shot.1','knockout_match.1','remaining_sec.1','distance_of_shot.1'],axis=1)

In [ ]:
dfenc.columns

In [ ]:
len(dfenc)

In [ ]:
sample_train = dfenc[['area_of_shot','is_goal']]

## Mean encoding 
x = sample_train.groupby(['area_of_shot'])['is_goal'].sum().reset_index()
x = x.rename(columns={"is_goal" : "area_shot_sum"})

y = sample_train.groupby(['area_of_shot'])['is_goal'].count().reset_index()
y = y.rename(columns={"is_goal" : "is_goal_count"})

z = pd.merge(x,y,on = 'area_of_shot',how = 'inner')
z['Target_Encoded_over_areashot'] = z['area_shot_sum']/z['is_goal_count']
#z.head()
#Title	Title_Survived_sum	Title_Survived_count	

In [ ]:
z = z[['area_of_shot','Target_Encoded_over_areashot']]

sample_train = pd.merge(sample_train,z,on = 'area_of_shot',how = 'left')
sample_train.isnull().sum()

In [ ]:
sample_train.drop('area_of_shot',inplace=True,axis=1)


In [ ]:
dfenc['area_of_shot']=sample_train['Target_Encoded_over_areashot']

In [ ]:
sample_train = dfenc[['shot_basics','is_goal']]

## Mean encoding 
x = sample_train.groupby(['shot_basics'])['is_goal'].sum().reset_index()
x = x.rename(columns={"is_goal" : "shot_basics_sum"})

y = sample_train.groupby(['shot_basics'])['is_goal'].count().reset_index()
y = y.rename(columns={"is_goal" : "is_goal_count1"})

z = pd.merge(x,y,on = 'shot_basics',how = 'inner')
z['Target_Encoded_over_shotbasics'] = z['shot_basics_sum']/z['is_goal_count1']
z
#Title	Title_Survived_sum	Title_Survived_count	

In [ ]:
sample_train = pd.merge(sample_train,z,on = 'shot_basics',how = 'left')
sample_train.head()



In [ ]:
dfenc['shot_basics']=sample_train['Target_Encoded_over_shotbasics']

In [ ]:
dfenc.isnull().sum()

In [ ]:
sample_train1 = dfenc[['range_of_shot','is_goal']]

## Mean encoding 
x1 = sample_train1.groupby(['range_of_shot'])['is_goal'].sum().reset_index()
x1 = x1.rename(columns={"is_goal" : "range_sum"})

y1 = sample_train1.groupby(['range_of_shot'])['is_goal'].count().reset_index()
y1 = y1.rename(columns={"is_goal" : "is_goal_count1"})

z1 = pd.merge(x1,y1,on = 'range_of_shot',how = 'inner')
z1['Target_Encoded_over_range'] = z1['range_sum']/z1['is_goal_count1']
z1
#Title	Title_Survived_sum	Title_Survived_count	

In [ ]:
z1 = z1[['range_of_shot','Target_Encoded_over_range']]

sample_train1 = pd.merge(sample_train1,z1,on = 'range_of_shot',how = 'left')
sample_train1.head()

In [ ]:
dfenc['range_of_shot']=sample_train1['Target_Encoded_over_range']

In [ ]:
sample_train1 = dfenc[['game_season','is_goal']]

## Mean encoding 
x1 = sample_train1.groupby(['game_season'])['is_goal'].sum().reset_index()
x1 = x1.rename(columns={"is_goal" : "season_sum"})

y1 = sample_train1.groupby(['game_season'])['is_goal'].count().reset_index()
y1 = y1.rename(columns={"is_goal" : "is_goal_count1"})

z1 = pd.merge(x1,y1,on = 'game_season',how = 'inner')
z1['Target_Encoded_over_season'] = z1['season_sum']/z1['is_goal_count1']
z1
#Title	Title_Survived_sum	Title_Survived_count	

In [ ]:
z1 = z1[['game_season','Target_Encoded_over_season']]

sample_train1 = pd.merge(sample_train1,z1,on = 'game_season',how = 'left')
sample_train1.head()

In [ ]:
dfenc['game_season']=sample_train1['Target_Encoded_over_season']

In [ ]:
sample_train1 = dfenc[['type_of_combined_shot','is_goal']]

## Mean encoding 
x1 = sample_train1.groupby(['type_of_combined_shot'])['is_goal'].sum().reset_index()
x1 = x1.rename(columns={"is_goal" : "shot_sum"})

y1 = sample_train1.groupby(['type_of_combined_shot'])['is_goal'].count().reset_index()
y1 = y1.rename(columns={"is_goal" : "is_goal_count1"})

z1 = pd.merge(x1,y1,on = 'type_of_combined_shot',how = 'inner')
z1['Target_Encoded_over_shot'] = z1['shot_sum']/z1['is_goal_count1']
z1
#Title	Title_Survived_sum	Title_Survived_count	

In [ ]:
z1 = z1[['type_of_combined_shot','Target_Encoded_over_shot']]

sample_train1 = pd.merge(sample_train1,z1,on = 'type_of_combined_shot',how = 'left')
sample_train1.head()

In [ ]:
dfenc['type_of_combined_shot']=sample_train1['Target_Encoded_over_shot']

In [ ]:
dfenc

In [ ]:
dfenc=dfenc.drop(['team_name','date_of_game','lat/lng'],axis=1)

In [ ]:
dff.head()

In [ ]:
submi_set=dff[dff.is_goal.isnull()]

In [ ]:
submi_set.drop(['shot_id_number','is_goal'],axis=1,inplace=True)

In [ ]:
submi_set.head()

In [ ]:
len(dff)

In [ ]:
len(submi_set)

In [ ]:
len(submission)

In [ ]:
dff.dropna(subset=['is_goal'],inplace=True)

In [ ]:
dff.drop('shot_id_number',axis=1,inplace=True)

In [ ]:
y=dff['is_goal']
len(y)

In [ ]:
dff.drop('is_goal',axis=1,inplace=True)

In [ ]:
dff.columns

In [ ]:
submi_set.columns

In [ ]:
from scipy import stats
dfout=dff[(np.abs(stats.zscore(dfenc)) < 3).all(axis=1)]

In [ ]:
subout=submi_set[(np.abs(stats.zscore(submi_set)) < 3).all(axis=1)]

In [ ]:
len(dfout)

In [ ]:
len(subout)

In [ ]:
len(dfout)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(dff, y, test_size = 0.15, random_state = 0)

In [ ]:
model = CatBoostClassifier(iterations=1500, learning_rate=0.02, l2_leaf_reg=4.5, depth=10, rsm=0.98, loss_function= 'Logloss', eval_metric='Logloss',use_best_model=True,random_seed=42)

In [ ]:
#model.fit(X_train,y_train,)
model.fit(X_train,y_train,eval_set=(X_test,y_test))

In [ ]:
from sklearn.feature_selection import RFECV
#using rfecv from feature_selection to see the fetaure importances to optimize our dataset
from sklearn.ensemble import RandomForestClassifier
clf_rf_4 =CatBoostClassifier(iterations=500, learning_rate=0.08, l2_leaf_reg=3.5, depth=9, rsm=0.98, loss_function= 'Logloss', eval_metric='Logloss',random_seed=42)
rfecv = RFECV(estimator=clf_rf_4, step=1, cv=4,scoring='accuracy')   #5-fold cross-validation
rfecv = rfecv.fit(X_train, y_train)

In [ ]:
print('Optimal number of features :', rfecv.n_features_)

print('Best features :', X_train.columns[rfecv.support_])

In [ ]:
dfbest=dff[['location_x','location_y','distance_of_shot','shot_basics','type_of_combined_shot','remaining_sec']]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(dfbest, y, test_size = 0.25, random_state = 0)

In [ ]:
model = CatBoostClassifier(iterations=2500, learning_rate=0.04, l2_leaf_reg=4.5, depth=10, rsm=0.98, loss_function= 'Logloss', eval_metric='Logloss',use_best_model=True,random_seed=42)

In [ ]:
#model.fit(X_train,y_train,)
model.fit(X_train,y_train,eval_set=(X_test,y_test))

In [ ]:
yprob=model.predict_proba(X_test)
yprob

In [ ]:
from sklearn.metrics import accuracy_score
ac_2 = accuracy_score(y_test,yprob)
print('Accuracy is: ',ac_2)